In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import psycopg2
import password as pw
import time

In [ ]:
# view_number_list = []
# for i in range(1, 58):
#     url = f'https://ani.gamer.com.tw/animeList.php?page={i}'
#     headers = {
#         'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
#     r = requests.get(url, headers=headers)
#     r.encoding = 'utf8'
#     if r.status_code == 200:
#         print(f'請求成功：{r.status_code}')
#     else:
#         print(f'請求失敗：{r.status_code}')
#     soup = BeautifulSoup(r.text, 'html.parser')
#     show_view_number = soup.select('.show-view-number > p')
#     for item in show_view_number:
#         view_number_list.append(item.text)
#     time.sleep(1)

In [ ]:
url = 'https://ani.gamer.com.tw/animeList.php?page=1'
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
r = requests.get(url, headers=headers)
r.encoding = 'utf8'
if r.status_code == 200:
    print(f'請求成功：{r.status_code}')
else:
    print(f'請求失敗：{r.status_code}')
intro_data = BeautifulSoup(r.text, 'html.parser')

In [ ]:
def download_data():
    url = 'https://ani.gamer.com.tw/animeList.php?page=2'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
    r = requests.get(url, headers=headers)
    r.encoding = 'utf8'
    if r.status_code == 200:
        print(f'請求成功：{r.status_code}')
    else:
        print(f'請求失敗：{r.status_code}')
    intro_data = BeautifulSoup(r.text, 'html.parser')
    anime_infos = intro_data.select('.theme-list-main')
    second_data = []
    for anime_info in anime_infos:
        show_view_number = anime_info.select_one('.show-view-number > p').text.strip()
        anime_name = anime_info.select_one('.theme-name').text.strip()
        anime_time = anime_info.select_one('.theme-time').text.strip().replace('年份：','')
        anime_episode = anime_info.select_one('.theme-number').text.strip().replace('共','').replace('集','')
        anime_link = 'https://ani.gamer.com.tw/' + anime_info['href']
        time.sleep(0.3)
        r1 = requests.get(anime_link, headers=headers)
        r1.encoding = 'utf8'
        detail_data = BeautifulSoup(r1.text, 'html.parser')
        acg_score = detail_data.select_one('.acg-score')
        star = acg_score.select_one('.score-overall-number').text.strip()
        rating_people = acg_score.select_one('.score-overall-people').text.strip().replace('人評價','')
        type_list = detail_data.select_one('.type-list')
        staff = []
        tags = []
        first_data = []
        for p in type_list.find_all('p'):
            staff.append(p.text)
        for li in type_list.select('.tag'):
            tags.append(li.text)
        infos = [anime_name,show_view_number,anime_time,anime_episode,anime_link,star,rating_people,staff[1],staff[2],staff[3]]
        first_data.append(infos)
        first_data.append(tags)
        second_data.append(first_data)
        print(f'動畫名:{anime_name}\n觀看數:{show_view_number}\n季度:{anime_time}\n集數:{anime_episode}\n動畫連結:{anime_link}\n{star}\n{rating_people}\n導演:{staff[1]}\n代理商:{staff[2]}\n製作廠商:{staff[3]}\n分類:{tags}\n')
    return second_data

In [ ]:
data = download_data()


In [ ]:
data

In [ ]:
data[0]

In [ ]:
def create_table(conn) -> None:
    cursor = conn.cursor()
    cursor.execute(
        '''
    CREATE TABLE  IF NOT EXISTS 巴哈姆特動畫瘋(
	id SERIAL,
	動畫名 TEXT NOT NULL,
	觀看數 TEXT NOT NULL,
	季度 TEXT NOT NULL,
	集數 TEXT NOT NULL,
	動畫連結 TEXT NOT NULL,
	星級 TEXT,
	評分人數 TEXT,
	導演監督 TEXT NOT NULL,
	台灣代理 TEXT NOT NULL,
	製作廠商 TEXT NOT NULL,
	作品分類1 TEXT DEFAULT NULL,
	作品分類2 TEXT DEFAULT NULL,
	作品分類3 TEXT DEFAULT NULL,
	作品分類4 TEXT DEFAULT NULL,
	作品分類5 TEXT DEFAULT NULL,
	PRIMARY KEY(id),
	UNIQUE(動畫名)
    )
        '''
    )
    cursor.close()
    conn.commit()

def insert_data(conn, infos:list[str], tags:list[str]) -> None:
	values = infos + tags
	if len(tags) == 0:
		sql = '''
			INSERT INTO 巴哈姆特動畫瘋
			(動畫名,觀看數,季度,集數,動畫連結,星級,評分人數,導演監督,台灣代理,製作廠商)
			VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
			ON CONFLICT (動畫名) DO NOTHING
			'''
	elif len(tags) == 1:
		sql = '''
			INSERT INTO 巴哈姆特動畫瘋
			(動畫名,觀看數,季度,集數,動畫連結,星級,評分人數,導演監督,台灣代理,製作廠商,作品分類1)
			VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
			ON CONFLICT (動畫名) DO NOTHING
			'''
	elif len(tags) == 2:
		sql = '''
			INSERT INTO 巴哈姆特動畫瘋
			(動畫名,觀看數,季度,集數,動畫連結,星級,評分人數,導演監督,台灣代理,製作廠商,作品分類1,作品分類2)
			VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
			ON CONFLICT (動畫名) DO NOTHING
			'''
	elif len(tags) == 3:
		sql = '''
			INSERT INTO 巴哈姆特動畫瘋
			(動畫名,觀看數,季度,集數,動畫連結,星級,評分人數,導演監督,台灣代理,製作廠商,作品分類1,作品分類2,作品分類3)
			VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
			ON CONFLICT (動畫名) DO NOTHING
			'''
	elif len(tags) == 4:
		sql = '''
			INSERT INTO 巴哈姆特動畫瘋
			(動畫名,觀看數,季度,集數,動畫連結,星級,評分人數,導演監督,台灣代理,製作廠商,作品分類1,作品分類2,作品分類3,作品分類4)
			VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
			ON CONFLICT (動畫名) DO NOTHING
			'''
	elif len(tags) == 5:
		sql = '''
			INSERT INTO 巴哈姆特動畫瘋
			(動畫名,觀看數,季度,集數,動畫連結,星級,評分人數,導演監督,台灣代理,製作廠商,作品分類1,作品分類2,作品分類3,作品分類4,作品分類5)
			VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
			ON CONFLICT (動畫名) DO NOTHING
			'''
	else:
		values = infos + [tags[0], tags[1], tags[2], tags[3], tags[4]]
		sql = '''
			INSERT INTO 巴哈姆特動畫瘋
			(動畫名,觀看數,季度,集數,動畫連結,星級,評分人數,導演監督,台灣代理,製作廠商,作品分類1,作品分類2,作品分類3,作品分類4,作品分類5)
			VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
			ON CONFLICT (動畫名) DO NOTHING
			'''
	cursor = conn.cursor()
	cursor.execute(sql, values)
	cursor.close()
	conn.commit()

In [ ]:
conn = psycopg2.connect(database=pw.DATABASE,
                        user=pw.USER,
                        password=pw.PASSWORD,
                        host=pw.HOST,
                        port=pw.PORT)

create_table(conn)
for anime_list in data:
    insert_data(conn,infos=anime_list[0],tags=anime_list[1])

In [ ]:
acg_score = detail_data.select_one('.acg-score')
star = acg_score.select_one('.score-overall-number')
rating_people = acg_score.select_one('.score-overall-people')
print(star.text, rating_people.text)

In [ ]:
type_list = detail_data.select_one('.type-list')

In [ ]:
print(type_list.find_all('span'))
print(type_list.find_all('p'))
print(type_list.select('.tag'))

In [ ]:
for p in type_list.find_all('p'):
    print(p.text)
for li in type_list.select('.tag'):
    print(li.text)

In [ ]:
new_view_number_list

In [ ]:
list = []

In [ ]:
wan_list = []

In [ ]:
for item in new_view_number_list:
    if '萬' in item:
        wan_list.append(item)

In [ ]:
list.remove('統計中')

In [ ]:
for i in range(len(list)):
    list[i] = int(list[i])

In [ ]:
list

In [ ]:
for i in range(len(wan_list)):
    n = wan_list[i].replace('萬', '')
    wan_list[i] = int(float(n) * 10000)

In [ ]:
view_number_list_in_number = wan_list + list

In [ ]:
view_number_list_in_number

In [ ]:
wan_list + list

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Your data
x = view_number_list_in_number

# Set up the figure and axes
plt.figure(figsize=(10, 6))
ax = sns.histplot(x, bins=1000)  # Adjust bins according to your data
ax.set_xticks([j for j in range(1, 21)])

plt.title('Histogram with Custom Axis Ticks')

plt.show()

In [ ]:
if item == '統計中':
            new_view_number_list.append(0)
            new_view_number_list.remove(item)
        elif int(item) < 1000:
            new_view_number_list.append(0)
            new_view_number_list.remove(item)
        elif 2000 > int(item) >= 1000:
            new_view_number_list.append(0.1)
            new_view_number_list.remove(item)
        elif 3000 > int(item) >= 2000:
            new_view_number_list.append(0.2)
            new_view_number_list.remove(item)
        elif 4000 > int(item) >= 3000:
            new_view_number_list.append(0.3)
            new_view_number_list.remove(item)
        elif 5000 > int(item) >= 4000:
            new_view_number_list.append(0.4)
            new_view_number_list.remove(item)
        elif 6000 > int(item) >= 5000:
            new_view_number_list.append(0.5)
            new_view_number_list.remove(item)
        elif 7000 > int(item) >= 6000:
            new_view_number_list.append(0.6)
            new_view_number_list.remove(item)
        elif 8000 > int(item) >= 7000:
            new_view_number_list.append(0.7)
            new_view_number_list.remove(item)
        elif 9000 > int(item) >= 8000:
            new_view_number_list.append(0.8)
            new_view_number_list.remove(item)
        elif 10000 > int(item) >= 9000:
            new_view_number_list.append(0.9)
            new_view_number_list.remove(item)